# CLIP 모델 구조 상세 분석

## 1. CLIP 모델 아키텍처 
- vision encoder (resnet or vision transformer 기반) 
- text encoder (transformer 기반, gpt-like 구조) 
- projection head (이미지와 텍스트 임베딩을 동일한 차원으로 변환)
- contrastive loss (이미지-텍스트 매칭 학습)

## 2. CLIP 주요 코드 분석
https://github.com/openai/CLIP

### (1) Vision Encoder (이미지 인코더)
- CLIP의 이미지 인코더는 ResNet 또는 Vision Transformer (ViT)를 사용

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import vit_b_32

class VisionTransformer(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        self.model = vit_b_32(pretrained=True)  # 미리 학습된 ViT 사용
        self.fc = nn.Linear(768, embed_dim)  # 최종 출력 차원을 512로 변환

    def forward(self, images):
        x = self.model(images)  # 이미지 인코딩
        x = self.fc(x)  # FC 레이어를 통해 차원 변환
        return x

- vit_b_32(pretrained=True): 사전 학습된 Vision Transformer(ViT) 사용
- self.fc = nn.Linear(768, embed_dim): ViT 출력 (768차원) → 512차원 변환
- 입력: 이미지 (Tensor)
- 출력: 512차원 이미지 임베딩 벡터

### (2) Text Encoder (텍스트 인코더)
- 텍스트 인코더는 Transformer 기반이고, GPT와 유사한 구조

In [ ]:
from transformers import RobertaModel

class TextEncoder(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        self.model = RobertaModel.from_pretrained("roberta-base")  # Pretrained Transformer
        self.fc = nn.Linear(768, embed_dim)  # 차원 축소

    def forward(self, input_ids, attention_mask):
        x = self.model(input_ids=input_ids, attention_mask=attention_mask)
        x = x.last_hidden_state[:, 0, :]  # CLS 토큰 사용
        x = self.fc(x)  # FC 레이어
        return x

- RobertaModel.from_pretrained("roberta-base"): 사전 학습된 RoBERTa 모델 사용
- x.last_hidden_state[:, 0, :]: 문장의 첫 번째 토큰(CLS 토큰) 사용
- self.fc = nn.Linear(768, embed_dim): 768차원 → 512차원 변환
- 입력 : 토큰화된 텍스트
- 출력 : 512차원 텍스트 임베딩

### (3) Projection Head (공통 임베딩 공간 매핑)
- 이미지와 텍스트 임베딩을 같은 공간으로 변환하기 위해 선형 변환을 사용.

In [ ]:
class ProjectionHead(nn.Module):
    def __init__(self, embed_dim=512, proj_dim=512):
        super().__init__()
        self.projection = nn.Linear(embed_dim, proj_dim)  # 512 -> 512 (동일 차원 변환)
        self.scale = nn.Parameter(torch.ones(1) * 0.07)  # Temperature scaling

    def forward(self, x):
        x = self.projection(x)
        x = x / x.norm(dim=-1, keepdim=True)  # L2 정규화
        return x * self.scale


- nn.Linear(embed_dim, proj_dim): 512차원 → 512차원 변환
- x / x.norm(dim=-1, keepdim=True): L2 정규화
- self.scale = nn.Parameter(torch.ones(1) * 0.07): Softmax 온도 파라미터

### (4) Contrastive Learning (대조 학습)
- CLIP은 Contrastive Learning을 사용해서 이미지와 텍스트를 서로 매칭하는 방식으로 학습.

In [ ]:
import torch.nn.functional as F

class CLIPLoss(nn.Module):
    def forward(self, image_features, text_features):
        # 이미지와 텍스트 간 내적 계산
        logits = image_features @ text_features.T  # Cosine Similarity
        labels = torch.arange(len(logits)).to(logits.device)

        # CrossEntropyLoss 적용 (이미지-텍스트 정답 매칭)
        loss = (F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels)) / 2
        return loss

- logits = image_features @ text_features.T: 이미지와 텍스트 벡터 간 내적(유사도 계산)
- labels = torch.arange(len(logits)): 정답 레이블 생성
- F.cross_entropy(logits, labels): 정답 텍스트와 가장 유사한 이미지 찾도록 학습

### (5) CLIP 전체 모델

In [ ]:
class CLIP(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        self.vision_encoder = VisionTransformer(embed_dim)
        self.text_encoder = TextEncoder(embed_dim)
        self.vision_proj = ProjectionHead(embed_dim)
        self.text_proj = ProjectionHead(embed_dim)
        self.loss_fn = CLIPLoss()

    def forward(self, images, input_ids, attention_mask):
        image_features = self.vision_proj(self.vision_encoder(images))
        text_features = self.text_proj(self.text_encoder(input_ids, attention_mask))
        loss = self.loss_fn(image_features, text_features)
        return loss, image_features, text_features